In [2]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer,
    ImageOverlay,
    Polyline,
    Polygon,
    Rectangle,
    Circle,
    CircleMarker,
    GeoJSON,
    GeomanDrawControl,
)

from traitlets import link

In [3]:
import json

with open("simple.geo.json") as f:
    data = json.load(f)

In [4]:
d1 = data.copy()
d2 = data.copy()

d1["features"] = d1["features"][:1]
d2["features"] = d2["features"][1:]


In [5]:
center = [46.475212657477016, 6.3198722284199675]
zoom = 9
m = Map(center=center, zoom=zoom, layout=dict(height="600px"))

g1 = GeoJSON(data=d1, pm_ignore=True, snap_ignore=True,     
            style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
           )

g2 = GeoJSON(data=d2, pm_ignore=True, snap_ignore=False)
# m.add(g1)
# m.add(g2)

draw_data = data["features"][1]
draw_data["properties"]["type"] = "circlemarker"
dc = GeomanDrawControl(
    marker={"markerStyle": {"color": "#FF0000"}},
    # rectangle={"pathOptions": {"color": "#FF0000"}},
    circlemarker={"pathOptions": {"color": "#FF0000"}},
    # circlemarker={},
    data=[draw_data]
)

# TODO add testing of other controls to make sure we didn't break them.


def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)


dc.on_draw(handle_draw)
m.add(dc)
m

Map(center=[46.475212657477016, 6.3198722284199675], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [6]:
import solara
import ipyleaflet
from typing import cast, Callable

In [7]:

@solara.component
def GeomanDrawControlComponent(
    *,
    on_draw=None,
    data=None,
    position="topright",
    current_mode=None,
    polygon=None,
    circle=None,
    marker=None,
    circlemarker=None,
    rectangle=None,
    polyline=None,
    text=None,
    edit=True,
    remove=True,
    drag=True,
    cut=True,
    rotate=True,
    hide_controls=False,
):
    if polyline is None:
        polyline = {}
    if rectangle is None:
        rectangle = {}
    if circlemarker is None:
        circlemarker = {}
    if marker is None:
        marker = {}
    if circle is None:
        circle = {}
    if polygon is None:
        polygon = {}
    if text is None:
        text = {}
    if data is None:
        data = []

    draw_control = ipyleaflet.GeomanDrawControl.element(
        data=data,
        # explicitly set these to empty dict to disable them
        circle=circle,
        marker=marker,
        circlemarker=circlemarker,
        polygon=polygon,
        rectangle=rectangle,
        polyline=polyline,
        text=text,
        position=position,
        edit=edit,
        remove=remove,
        current_mode=current_mode,
        drag=drag,
        cut=cut,
        rotate=rotate,
        hide_controls=hide_controls,
    )

    def add_draw_handler():
        if on_draw is None:
            return
        widget = cast(ipyleaflet.GeomanDrawControl, solara.get_widget(draw_control))
        if isinstance(on_draw, Callable):
            widget.on_draw(on_draw)
        else:
            for callback in on_draw:
                widget.on_draw(callback)

        def cleanup():
            widget.on_draw(on_draw, remove=True)

        return cleanup

    solara.use_effect(add_draw_handler, dependencies=[])

    return draw_control

In [8]:
data["features"][1]

{'type': 'Feature',
 'properties': {'type': 'circlemarker'},
 'geometry': {'coordinates': [6.146144956875389, 46.4506138274553],
  'type': 'Point'}}

In [9]:
@solara.component
def EditMap(edit):
    center = [46.475212657477016, 6.3198722284199675]
    zoom = 9
    
    # edit, set_edit = solara.use_state(True)
    # edit = solara.reactive(True)
    
    g1 = GeoJSON.element(
        data=d1, pm_ignore=True, snap_ignore=True,
        style={
            'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1, 'color': '#666666', 'fillColor': '#666666'
        },
    )
    
    g2 = GeoJSON.element(
        data=d2, pm_ignore=True, snap_ignore=True,
        style={
            'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1, 'color': '#666666', 'fillColor': '#666666'
        }
    )
    
    dc = GeomanDrawControlComponent(
        marker={"shapeOptions": {"color": "#FF0000"}},
        # rectangle={"shapeOptions": {"color": "#FF0000"}},
        rectangle={},
        # circle={"shapeOptions": {"color": "#FF0000"}},
        circle={},
        circlemarker={"shapeOptions": {"color": "#FF0000"}},
        data=[data["features"][1]],
        edit=edit.value,
        on_draw=handle_draw
    )
    
    layers = [
        g1, g2
    ]
    
    controls = [dc]
    
    ipyleaflet.Map.element(
                layout=dict(height="400px"),
                layers=layers,
                center=center,
                zoom=zoom,
                controls=controls,
                # zoom_control=False,
                scroll_wheel_zoom=True,
            ).meta(ref="map-widget")

In [10]:
edit.value = not edit.value

NameError: name 'edit' is not defined

In [11]:
edit = solara.reactive(True)
EditMap(edit)

Cannot show ipywidgets in text